# 02 - Feature Engineering

**Objectif** : Créer de nouvelles variables (features) pour améliorer le modèle.

---

## Questions à se poser :
1. Quelles informations temporelles peut-on extraire de la date ?
2. Y a-t-il des features Landsat qu'on n'utilise pas encore ?
3. Peut-on créer des combinaisons de features (ratios, produits) ?
4. Les nouvelles features sont-elles corrélées avec les targets ?

---

## Programme du notebook :
1. Charger les données
2. Créer des features temporelles (mois, saison)
3. Explorer les autres features Landsat disponibles
4. Créer de nouveaux indices spectraux (ratios)
5. Vérifier si les nouvelles features sont utiles
6. Créer une fonction réutilisable

---
## 1. Imports et chargement

In [ ]:
# Imports de base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration des graphiques
plt.rcParams['figure.figsize'] = (10, 6)

# Pour ignorer les warnings
import warnings
warnings.filterwarnings('ignore')

# Accéder au dossier src/
import sys
sys.path.append('..')

from src.paths import WATER_QUALITY_FILE, LANDSAT_FILE, TERRACLIMATE_FILE
from src.data.load_data import load_all
from src.config import TARGETS, BENCHMARK_FEATURES, LANDSAT_FEATURES

print("Imports OK!")

In [ ]:
# Charger les données
X, y, site_ids, df = load_all(
    str(WATER_QUALITY_FILE),
    str(LANDSAT_FILE),
    str(TERRACLIMATE_FILE)
)

---
## 2. Features temporelles

**Pourquoi ?** La qualité de l'eau varie selon les saisons (pluie, température, etc.)

In [ ]:
# Extraire le mois et l'année de la date
df['year'] = df['Sample Date'].dt.year
df['month'] = df['Sample Date'].dt.month
df['day_of_year'] = df['Sample Date'].dt.dayofyear

print("Nouvelles colonnes temporelles créées!")
df[['Sample Date', 'year', 'month', 'day_of_year']].head()

In [ ]:
# Créer une variable "saison"
# Note: Afrique du Sud = hémisphère sud (saisons inversées)
def get_season(month):
    if month in [12, 1, 2]:
        return 'summer'  # Été austral
    elif month in [3, 4, 5]:
        return 'autumn'
    elif month in [6, 7, 8]:
        return 'winter'
    else:
        return 'spring'

df['season'] = df['month'].apply(get_season)
print("Répartition par saison :")
print(df['season'].value_counts())

In [ ]:
# Visualiser l'effet de la saison sur les targets
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, target in enumerate(TARGETS):
    ax = axes[i]
    df.boxplot(column=target, by='season', ax=ax)
    ax.set_title(target)
    ax.set_xlabel('Saison')

plt.suptitle('')  # Enlever le titre automatique
plt.tight_layout()
plt.show()

---
## 3. Utiliser plus de features Landsat

Le benchmark n'utilise que `swir22`, `NDMI`, `MNDWI`. Il y a d'autres bandes disponibles!

In [ ]:
# Quelles features Landsat sont disponibles ?
print("Features Landsat disponibles :")
for f in LANDSAT_FEATURES:
    if f in df.columns:
        print(f"  - {f} : OK")
    else:
        print(f"  - {f} : MANQUANTE")

In [ ]:
# Corrélation des features Landsat avec les targets
landsat_in_df = [f for f in LANDSAT_FEATURES if f in df.columns]

corr = df[landsat_in_df + TARGETS].corr()
corr_subset = corr.loc[landsat_in_df, TARGETS]

plt.figure(figsize=(10, 6))
sns.heatmap(corr_subset, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Corrélation Features Landsat vs Targets')
plt.show()

---
## 4. Créer de nouveaux indices spectraux

Les indices spectraux sont des combinaisons de bandes qui ont une signification physique.

In [ ]:
# Exemple : ratio entre bandes
# Ces ratios peuvent capturer des informations sur la turbidité, la chlorophylle, etc.

if 'nir' in df.columns and 'green' in df.columns:
    # Ratio NIR/Green (sensible à la végétation/algues)
    df['nir_green_ratio'] = df['nir'] / (df['green'] + 0.0001)  # +0.0001 pour éviter division par 0
    print("Feature 'nir_green_ratio' créée!")

if 'swir16' in df.columns and 'swir22' in df.columns:
    # Ratio entre les deux SWIR
    df['swir_ratio'] = df['swir16'] / (df['swir22'] + 0.0001)
    print("Feature 'swir_ratio' créée!")

---
## 5. Résumé des features créées

In [ ]:
# Liste des nouvelles features
new_features = ['year', 'month', 'day_of_year', 'season']

if 'nir_green_ratio' in df.columns:
    new_features.append('nir_green_ratio')
if 'swir_ratio' in df.columns:
    new_features.append('swir_ratio')

print("Nouvelles features créées :")
for f in new_features:
    print(f"  - {f}")

In [ ]:
# Tester la corrélation des nouvelles features numériques avec les targets
numeric_new_features = [f for f in new_features if f != 'season' and f in df.columns]

if numeric_new_features:
    corr = df[numeric_new_features + TARGETS].corr()
    corr_subset = corr.loc[numeric_new_features, TARGETS]
    
    plt.figure(figsize=(10, 6))
    sns.heatmap(corr_subset, annot=True, cmap='coolwarm', center=0, fmt='.2f')
    plt.title('Corrélation Nouvelles Features vs Targets')
    plt.show()

---
## 6. Fonction pour créer toutes les features

On regroupe tout dans une fonction réutilisable.

In [ ]:
def create_features(df):
    """
    Crée toutes les features supplémentaires.
    
    Paramètres:
        df : DataFrame avec les données brutes
    
    Retourne:
        DataFrame avec les nouvelles features
    """
    df = df.copy()
    
    # Features temporelles
    if 'Sample Date' in df.columns:
        df['year'] = df['Sample Date'].dt.year
        df['month'] = df['Sample Date'].dt.month
        df['day_of_year'] = df['Sample Date'].dt.dayofyear
        
        # Saison (hémisphère sud)
        def get_season(month):
            if month in [12, 1, 2]: return 'summer'
            elif month in [3, 4, 5]: return 'autumn'
            elif month in [6, 7, 8]: return 'winter'
            else: return 'spring'
        
        df['season'] = df['month'].apply(get_season)
    
    # Ratios spectraux
    if 'nir' in df.columns and 'green' in df.columns:
        df['nir_green_ratio'] = df['nir'] / (df['green'] + 0.0001)
    
    if 'swir16' in df.columns and 'swir22' in df.columns:
        df['swir_ratio'] = df['swir16'] / (df['swir22'] + 0.0001)
    
    return df

print("Fonction create_features() définie!")

---
## 7. Prochaine étape

Maintenant qu'on a créé des features, on peut :
1. Tester si elles améliorent le modèle (notebook 03)
2. Créer encore plus de features si besoin

### Idées pour aller plus loin :
- Ajouter d'autres variables TerraClimate (température, précipitations...)
- Créer des features de lag (valeur du mois précédent)
- Encoder la saison en sin/cos (encodage cyclique)

→ **Notebook 03 : Modélisation**